# CRNN Models with Augmentation and Ensemble Method
# Baseline Model

In [0]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
# setup
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import math
from google.colab import drive
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt

from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Permute, Bidirectional, BatchNormalization, Conv2D,  Dense, Dropout, ELU, Flatten, MaxPool2D, TimeDistributed
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import GridSearchCV

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Project247/'

X_test = np.load(root_path+"X_test.npy")
y_test = np.load(root_path+"y_test.npy")
person_train_valid = np.load(root_path+"person_train_valid.npy")
X_train_valid = np.load(root_path+"X_train_valid.npy")
y_train_valid = np.load(root_path+"y_train_valid.npy")
person_test = np.load(root_path+"person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

# test GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


TensorFlow 2.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
Found GPU at: /device:GPU:0


In [0]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)


In [0]:
# split function for pices of data
def split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = False, use_cv = False, cv = 5, size = 400, step = 20, end = 1000, printSize = False):
  y_test_norm = y_test - np.min(y_test)
  y_train_valid_norm = y_train_valid - np.min(y_train_valid)
  # split train and validation 80:20 (front-rare)
  if use_cv:
    X_train_valid_split = np.array( np.array_split(X_train_valid, cv) )
    X_train = np.concatenate(X_train_valid_split[0:cv-1])
    X_valid = X_train_valid_split[cv-1]
    y_train_valid_split = np.array( np.array_split(y_train_valid_norm, cv) )
    y_train = np.concatenate(y_train_valid_split[0:cv-1])
    y_valid = y_train_valid_split[cv-1]
  else:
    X_train = X_train_valid
    X_valid = X_train_valid[0:2]
    y_train = y_train_valid_norm
    y_valid = y_train_valid_norm[0:2]
  

  # make time be the index (1000, 22, 443)
  X_test_pcs_rsp = X_test.transpose(2,1,0)
  X_train_pcs_rsp = X_train.transpose(2,1,0)
  X_valid_pcs_rsp = X_valid.transpose(2,1,0)
  times = math.ceil( (end-size) /step )

  # split timestamp into small pieces (45, 100, 22, 443)
  if test_split:
    X_test_pcs_rsp = np.array([X_test_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  else:
    X_test_pcs_rsp = X_test_pcs_rsp[0 : size]
  X_train_pcs_rsp =  np.array([X_train_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  X_valid_pcs_rsp =  np.array([X_valid_pcs_rsp[i : i + size] for i in range(0, end-size, step)])
  
  # make samples be the index (45, 443, 22, 100)
  if test_split:
    X_test_pcs_rsp = X_test_pcs_rsp.transpose(0,3,2,1)
  else:
    X_test_pcs_rsp = X_test_pcs_rsp.transpose(2,1,0)
  X_train_pcs_rsp = X_train_pcs_rsp.transpose(0,3,2,1)
  X_valid_pcs_rsp = X_valid_pcs_rsp.transpose(0,3,2,1)
  
  # combine all samples together (45*443, 22, 100)
  if test_split:
    X_test_pcs_rsp = np.concatenate(X_test_pcs_rsp, axis=0)
  X_train_pcs_rsp = np.concatenate(X_train_pcs_rsp, axis=0)
  X_valid_pcs_rsp = np.concatenate(X_valid_pcs_rsp, axis=0)
  
  # repeat n times of the whole y array
  if test_split:
    y_test_norm_pcs = np.tile(y_test_norm, times)
  else:
    y_test_norm_pcs = y_test_norm
  y_train_norm_pcs = np.tile(y_train, times)
  y_valid_norm_pcs = np.tile(y_valid, times)
  if printSize:
      print(X_test_pcs_rsp.shape)
      print(X_train_pcs_rsp.shape)
      print(X_valid_pcs_rsp.shape)
      print(len(y_test_norm_pcs))
      print(len(y_train_norm_pcs))
      print(len(y_valid_norm_pcs))
  y_test_norm_pcs_b = to_categorical(y_test_norm_pcs)
  y_train_norm_pcs_b = to_categorical(y_train_norm_pcs)
  y_valid_norm_pcs_b = to_categorical(y_valid_norm_pcs)
  return X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs_b, y_valid_norm_pcs_b, y_test_norm_pcs_b

In [0]:
def crnn(kernel_initializer, time_period=1000, l1_l = 0, l2_l = 0.01, drop_ratio = 0.4, step = 0.001 ,decay_r = 0.01, optim = 'Adam'):
    model = Sequential()

    model.add(Conv2D(16, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=l1_l, l2=l2_l), input_shape=(22, time_period, 1), data_format='channels_last'))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))

    model.add(Conv2D(32, kernel_size=(21, 1), strides=1, padding='valid', kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))

    model.add(Conv2D(64, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
    model.add(ELU())
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D((1, 4)))
    
    if time_period >= 300:
      model.add(Conv2D(128, kernel_size=(1, 10), strides=1, padding='valid', kernel_regularizer=L1L2(l1=l1_l, l2=l2_l)))
      model.add(ELU())
      model.add(BatchNormalization(axis=1))
      model.add(MaxPool2D((1, 4)))
      model.add(Dropout(drop_ratio))

    model.add(Permute((2, 3, 1)))
    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(CuDNNLSTM(128, kernel_initializer=kernel_initializer, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(64, kernel_initializer=kernel_initializer, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(32, kernel_initializer=kernel_initializer)))
    model.add(Dropout(drop_ratio))
    model.add(Dense(4, kernel_regularizer=L1L2(l1=l1_l, l2=l2_l), activation='softmax'))
    if optim == 'Adam':
      model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=step, epsilon=1e-8, decay=decay_r), metrics=[categorical_accuracy])
    elif optim == 'RMSprop':
      model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=step,rho=0.9), metrics=[categorical_accuracy])
    elif optim == 'Nesterov':
      model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=step, decay=1e-6, momentum=0.9, nesterov=True), metrics=[categorical_accuracy])
      
    return model

In [0]:
all_tr = []
all_te = []
best_step,best_epoch,best_batch_s, best_opt = 0.0,0.0,0.0, ''
epoch_num = [40]
batch_size = [40]

optimize = ['Adam']
step_s = 0.001

for epoch_s in epoch_num:
  for batch_s in batch_size:
    for opt in optimize:
      train_scores = []
      test_scores = []
      for time in [900]:
        size = time
        step = 50
        init = GlorotUniform(seed=42)     
        CRNNmodel = crnn(init, time_period = time, l1_l = 0 ,l2_l = 0.01, step = step_s, drop_ratio = 0.4, decay_r = 0.01, optim = opt)
        CRNNmodel.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 891, 16)       176       
_________________________________________________________________
elu (ELU)                    (None, 22, 891, 16)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 891, 16)       88        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 891, 32)        10784     
_________________________________________________________________
elu_1 (ELU)                  (None, 2, 891, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 891, 32)        8         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 222, 32)        0

Without Data Augmentation

In [0]:
all_tr = []
all_te = []
best_step,best_epoch,best_batch_s, best_opt = 0.0,0.0,0.0, ''
epoch_num = [40]
batch_size = [40]
#optimize = ['Nesterov']
optimize = ['Adam']

In [0]:
step_s = 0.001

for epoch_s in epoch_num:
  for batch_s in batch_size:
    for opt in optimize:
      train_scores = []
      test_scores = []
      for time in [900]:
        size = time
        step = 100
        end = 1000
        X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
          split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
        print('Time: %d, batch_size: %d, epoch_num: %d, step: %.6f, optimizer: %s'%(time,batch_s,epoch_s,step_s,opt))
        init = GlorotUniform(seed=42)
        # preprocess data
        X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
        X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
        X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
        CRNNmodel = crnn(init, time_period = time, l1_l = 0 ,l2_l = 0, step = step_s, drop_ratio = 0.4, decay_r = 0.01, optim = opt)
        with tf.device('/device:GPU:0'):
          # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
          history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch_s, batch_size=batch_s, verbose=1,validation_data=(X_valid_cur, y_valid_norm_pcs))
        # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
        val_score = history.history['val_categorical_accuracy'][-1]
        print(val_score)

(443, 22, 900)
(1692, 22, 900)
(423, 22, 900)
443
1692
423
Time: 900, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
Train on 1692 samples, validate on 423 samples
Epoch 1/40
1692/1692 [==============================] - 13s 8ms/sample - loss: 1.4298 - categorical_accuracy: 0.2346 - val_loss: 1.3933 - val_categorical_accuracy: 0.2506
Epoch 2/40
1692/1692 [==============================] - 1s 764us/sample - loss: 1.3911 - categorical_accuracy: 0.2665 - val_loss: 1.4149 - val_categorical_accuracy: 0.2506
Epoch 3/40
1692/1692 [==============================] - 1s 780us/sample - loss: 1.3728 - categorical_accuracy: 0.3061 - val_loss: 1.3803 - val_categorical_accuracy: 0.2884
Epoch 4/40
1692/1692 [==============================] - 1s 781us/sample - loss: 1.3452 - categorical_accuracy: 0.3333 - val_loss: 1.3702 - val_categorical_accuracy: 0.3144
Epoch 5/40
1692/1692 [==============================] - 1s 783us/sample - loss: 1.3225 - categorical_accuracy: 0.3647 - val_loss: 1.3

In [0]:
test_acc = CRNNmodel.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

443/443 - 0s - loss: 1.4596 - categorical_accuracy: 0.5756
test_acc [1.4595818350062144, 0.5756208]


With Data Augmentation

In [0]:
all_tr = []
all_te = []
best_step,best_epoch,best_batch_s, best_opt = 0.0,0.0,0.0, ''
epoch_num = [40]
batch_size = [40]
#optimize = ['Nesterov']
optimize = ['Adam']

In [0]:
step_s = 0.001

for epoch_s in epoch_num:
  for batch_s in batch_size:
    for opt in optimize:
      train_scores = []
      test_scores = []
      for time in [900]:
        size = time
        step = 50
        end = int(size + step*3)    # gives 3x
        X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
          split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
        print('Time: %d, batch_size: %d, epoch_num: %d, step: %.6f, optimizer: %s'%(time,batch_s,epoch_s,step_s,opt))
        init = GlorotUniform(seed=42)
        # preprocess data
        X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
        X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
        X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
        CRNNmodel = crnn(init, time_period = time, l1_l = 0 ,l2_l = 0, step = step_s, drop_ratio = 0.4, decay_r = 0.01, optim = opt)
        with tf.device('/device:GPU:0'):
          # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
          history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch_s, batch_size=batch_s, verbose=1,validation_data=(X_valid_cur, y_valid_norm_pcs))
        # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
        val_score = history.history['val_categorical_accuracy'][-1]
        print(val_score)

(1329, 22, 900)
(5076, 22, 900)
(1269, 22, 900)
1329
5076
1269
Time: 900, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
Train on 5076 samples, validate on 1269 samples
Epoch 1/40
5076/5076 [==============================] - 8s 2ms/sample - loss: 1.3872 - categorical_accuracy: 0.2857 - val_loss: 1.3970 - val_categorical_accuracy: 0.2821
Epoch 2/40
5076/5076 [==============================] - 4s 778us/sample - loss: 1.3225 - categorical_accuracy: 0.3621 - val_loss: 1.3197 - val_categorical_accuracy: 0.3822
Epoch 3/40
5076/5076 [==============================] - 4s 764us/sample - loss: 1.2094 - categorical_accuracy: 0.4687 - val_loss: 1.2060 - val_categorical_accuracy: 0.4563
Epoch 4/40
5076/5076 [==============================] - 4s 764us/sample - loss: 1.0907 - categorical_accuracy: 0.5451 - val_loss: 1.1496 - val_categorical_accuracy: 0.5020
Epoch 5/40
5076/5076 [==============================] - 4s 766us/sample - loss: 0.9999 - categorical_accuracy: 0.5973 - val_loss:

In [0]:
test_acc = CRNNmodel.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

1329/1329 - 1s - loss: 1.5080 - categorical_accuracy: 0.6110
test_acc [1.5080067829766428, 0.6109857]


# Determine the Optimal Window Size
Without Data Augmentation

In [0]:
all_tr = []
all_te = []
best_step,best_epoch,best_batch_s, best_opt = 0.0,0.0,0.0, ''
epoch_num = [40]
batch_size = [40]
#optimize = ['Nesterov']
optimize = ['Adam']

In [0]:
step_s = 0.001
val_list = []

for epoch_s in epoch_num:
  for batch_s in batch_size:
    for opt in optimize:
      train_scores = []
      test_scores = []
      for time in range(100,1000,100):
        size = time
        step = 1000-size
        end = 1000
        X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
          split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
        print('Time: %d, batch_size: %d, epoch_num: %d, step: %.6f, optimizer: %s'%(time,batch_s,epoch_s,step_s,opt))
        init = GlorotUniform(seed=42)
        # preprocess data
        X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
        X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
        X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
        CRNNmodel = crnn(init, time_period = time, l1_l = 0 ,l2_l = 0.01, step = step_s, drop_ratio = 0.4, decay_r = 0.01, optim = opt)
        with tf.device('/device:GPU:0'):
          # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
          history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch_s, batch_size=batch_s, verbose=0, validation_freq=[epoch_s], validation_data=(X_valid_cur, y_valid_norm_pcs))
        # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
        val_acc = history.history['val_categorical_accuracy'][-1]
        train_acc = history.history['categorical_accuracy'][-1]
        val_list.append(val_acc)
        print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)

Time: 100, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 100 train_acc 0.99940896 val_acc 0.46335697
Time: 200, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 200 train_acc 0.9964539 val_acc 0.5035461
Time: 300, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 300 train_acc 0.96631205 val_acc 0.5177305
Time: 400, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 400 train_acc 0.9615839 val_acc 0.6359338
Time: 500, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 500 train_acc 0.9674941 val_acc 0.6099291
Time: 600, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 600 train_acc 0.9444444 val_acc 0.61702126
Time: 700, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 700 train_acc 0.9550827 val_acc 0.61938536
Time: 800, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 800 t

With Data Augmentation

In [0]:
all_tr = []
all_te = []
best_step,best_epoch,best_batch_s, best_opt = 0.0,0.0,0.0, ''
epoch_num = [40]
batch_size = [40]
#optimize = ['Nesterov']
optimize = ['Adam']

In [0]:
step_s = 0.001
val_list = []

for epoch_s in epoch_num:
  for batch_s in batch_size:
    for opt in optimize:
      train_scores = []
      test_scores = []
      for time in range(100,1000,100):
        size = time
        step = 30
        end = int(size + step*3)    # gives 3x
        X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
          split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
        print('Time: %d, batch_size: %d, epoch_num: %d, step: %.6f, optimizer: %s'%(time,batch_s,epoch_s,step_s,opt))
        init = GlorotUniform(seed=42)
        # preprocess data
        X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
        X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
        X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
        CRNNmodel = crnn(init, time_period = time, l1_l = 0 ,l2_l = 0.01, step = step_s, drop_ratio = 0.4, decay_r = 0.01, optim = opt)
        with tf.device('/device:GPU:0'):
          # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
          history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch_s, batch_size=batch_s, verbose=0, validation_freq=[epoch_s], validation_data=(X_valid_cur, y_valid_norm_pcs))
        # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
        val_acc = history.history['val_categorical_accuracy'][-1]
        train_acc = history.history['categorical_accuracy'][-1]
        val_list.append(val_acc)
        print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)

Time: 100, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 100 train_acc 0.9858156 val_acc 0.41213554
Time: 200, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 200 train_acc 0.999803 val_acc 0.534279
Time: 300, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 300 train_acc 0.93026006 val_acc 0.55476755
Time: 400, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 400 train_acc 0.9724192 val_acc 0.6572104
Time: 500, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 500 train_acc 0.96926713 val_acc 0.64460206
Time: 600, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 600 train_acc 0.96907014 val_acc 0.66351455
Time: 700, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 700 train_acc 0.9708432 val_acc 0.68715525
Time: 800, batch_size: 40, epoch_num: 40, step: 0.001000, optimizer: Adam
time window: 800 

# Grid Search for Optimal Hyperparameters
We pick time window sizes of 700 and 900

In [0]:
# For time window of 700
epoch_list = [40]
batch_list = [48]
opt_list = ['Adam','Nesterov','RMSprop']
reg_name_list = ['l2','l1']
reg_list = [0.01, 0.001]
step_list = [0.001]
decay_list = [0.01, 0.001]
size_list = [700]
total_iter = len(epoch_list)*len(batch_list)*len(opt_list)*len(reg_name_list)*len(reg_list)*len(step_list)*len(decay_list)*len(size_list)

In [0]:
acc_list = []
param_list = []
step = 10
iter=0

for size in size_list:
  end = int(size + step*3)    # gives 3x
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  init = GlorotUniform(seed=42)
  # preprocess data
  X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
  X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
  X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
  for reg_name in reg_name_list:
    if reg_name == 'l1':
      l1_l_list = reg_list
      l2_l_list = [0]
    elif reg_name == 'l2':
      l2_l_list = reg_list
      l1_l_list = [0]
    else:
      l1_l_list = [0]
      l2_l_list = [0]
    for l1_l in l1_l_list:
      for l2_l in l2_l_list:
        for opt in opt_list:
          for step in step_list:
            for decay in decay_list:
              for batch in batch_list:
                for epoch in epoch_list:
                  iter += 1
                  print('Iteration',iter,'of',total_iter)
                  CRNNmodel = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
                  with tf.device('/device:GPU:0'):
                    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
                    history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_freq=[epoch], validation_data=(X_valid_cur, y_valid_norm_pcs))
                  # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
                  val_acc = history.history['val_categorical_accuracy'][-1]
                  train_acc = history.history['categorical_accuracy'][-1]
                  print('size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
                  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data
                  param_list.append(['size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch])
                  acc_list.append(val_acc)


Iteration 1 of 24
size 700 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.01 batch size 48 epochs 40
time window: 700 train_acc 0.98325455 val_acc 0.67769897
Iteration 2 of 24
size 700 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.001 batch size 48 epochs 40
time window: 700 train_acc 0.9921198 val_acc 0.643814
Iteration 3 of 24
size 700 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.01 batch size 48 epochs 40
time window: 700 train_acc 0.95981085 val_acc 0.70133966
Iteration 4 of 24
size 700 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 700 train_acc 0.9523247 val_acc 0.6761229
Iteration 5 of 24
size 700 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.01 batch size 48 epochs 40
time window: 700 train_acc 0.97399527 val_acc 0.571316
Iteration 6 of 24
size 700 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.001 batch size 48 epochs 40
time wind

In [0]:
# Output the top # of parameters
top_num = 4
acc_sorted, para_sorted = zip(*sorted(zip(acc_list, param_list),reverse=True))
for j in range(top_num):
  print('#',j+1,'accuracy')
  print(para_sorted[j])
  print(acc_sorted[j])

In [0]:
# For time window of 900
epoch_list = [40]
batch_list = [48]
opt_list = ['Adam','Nesterov','RMSprop']
reg_name_list = ['l2','l1']
reg_list = [0.01, 0.001]
step_list = [0.001]
decay_list = [0.01, 0.001]
size_list = [900]
total_iter = len(epoch_list)*len(batch_list)*len(opt_list)*len(reg_name_list)*len(reg_list)*len(step_list)*len(decay_list)*len(size_list)

In [0]:
acc_list = []
param_list = []
step = 10
iter=0

for size in size_list:
  end = int(size + step*3)    # gives 3x
  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  init = GlorotUniform(seed=42)
  # preprocess data
  X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
  X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
  X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
  for reg_name in reg_name_list:
    if reg_name == 'l1':
      l1_l_list = reg_list
      l2_l_list = [0]
    elif reg_name == 'l2':
      l2_l_list = reg_list
      l1_l_list = [0]
    else:
      l1_l_list = [0]
      l2_l_list = [0]
    for l1_l in l1_l_list:
      for l2_l in l2_l_list:
        for opt in opt_list:
          for step in step_list:
            for decay in decay_list:
              for batch in batch_list:
                for epoch in epoch_list:
                  iter += 1
                  print('Iteration',iter,'of',total_iter)
                  CRNNmodel = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
                  with tf.device('/device:GPU:0'):
                    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
                    history = CRNNmodel.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_freq=[epoch], validation_data=(X_valid_cur, y_valid_norm_pcs))
                  # train_score = CRNNmodel.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
                  val_acc = history.history['val_categorical_accuracy'][-1]
                  train_acc = history.history['categorical_accuracy'][-1]
                  print('size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
                  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data
                  param_list.append(['size',size,'reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch])
                  acc_list.append(val_acc)


Iteration 1 of 24
size 900 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.01 batch size 48 epochs 40
time window: 900 train_acc 0.9812845 val_acc 0.6745469
Iteration 2 of 24
size 900 reg name l2 reg value 0 0.01 opt name Adam step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.9895587 val_acc 0.606777
Iteration 3 of 24
size 900 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.01 batch size 48 epochs 40
time window: 900 train_acc 0.9493696 val_acc 0.68085104
Iteration 4 of 24
size 900 reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.9470055 val_acc 0.7100079
Iteration 5 of 24
size 900 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.01 batch size 48 epochs 40
time window: 900 train_acc 0.9773444 val_acc 0.54688734
Iteration 6 of 24
size 900 reg name l2 reg value 0 0.01 opt name RMSprop step 0.001 decay 0.001 batch size 48 epochs 40
time window

In [0]:
# Output the top # of parameters
top_num = 4
acc_sorted, para_sorted = zip(*sorted(zip(acc_list, param_list),reverse=True))
for j in range(top_num):
  print('#',j+1,'accuracy')
  print(para_sorted[j])
  print(acc_sorted[j])

# 1 accuracy
['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]
0.7115839
# 2 accuracy
['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]
0.7100079
# 3 accuracy
['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 40]
0.6958235
# 4 accuracy
['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.01, 'batch size', 48, 'epochs', 40]
0.68085104


# Run the Optimal Model and Evaluate the Testing Accuracy
We use higher volume of data augmentation, from 3x to 10x.

In [0]:
# ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

size = 900
step = 10
end = int(size + step*10)    # gives 10x

X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
  split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
init = GlorotUniform(seed=42)
# preprocess data
X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
reg_name = 'l1'
l1_l = 0.001
l2_l = 0
opt = 'Nesterov'
step = 0.001
decay = 0.001
batch = 48
epoch = 40

CRNNopt = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
with tf.device('/device:GPU:0'):
  # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
  history = CRNNopt.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=1, validation_data=(X_valid_cur, y_valid_norm_pcs))
# train_score = CRNNopt.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
val_acc = history.history['val_categorical_accuracy'][-1]
train_acc = history.history['categorical_accuracy'][-1]
print('reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data


(4430, 22, 900)
(16920, 22, 900)
(4230, 22, 900)
4430
16920
4230
Train on 16920 samples, validate on 4230 samples
Epoch 1/40
16920/16920 [==============================] - 30s 2ms/sample - loss: 4.7543 - categorical_accuracy: 0.2983 - val_loss: 4.5502 - val_categorical_accuracy: 0.3258
Epoch 2/40
16920/16920 [==============================] - 12s 719us/sample - loss: 4.3333 - categorical_accuracy: 0.3548 - val_loss: 4.1452 - val_categorical_accuracy: 0.3820
Epoch 3/40
16920/16920 [==============================] - 12s 713us/sample - loss: 3.9008 - categorical_accuracy: 0.4365 - val_loss: 3.7427 - val_categorical_accuracy: 0.4426
Epoch 4/40
16920/16920 [==============================] - 12s 711us/sample - loss: 3.4459 - categorical_accuracy: 0.5387 - val_loss: 3.2637 - val_categorical_accuracy: 0.5508
Epoch 5/40
16920/16920 [==============================] - 12s 714us/sample - loss: 3.0015 - categorical_accuracy: 0.6171 - val_loss: 2.9248 - val_categorical_accuracy: 0.5719
Epoch 6/40
16

In [0]:
test_acc = CRNNopt.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

4430/4430 - 8s - loss: 2.2064 - categorical_accuracy: 0.6442
test_acc [2.206397454679416, 0.6442438]


Record Scores for Ensembleing

In [0]:
predictions_1 = CRNNopt.predict(X_test_cur)

### Second Model

In [0]:
# ['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

size = 900
step = 10
end = int(size + step*10)    # gives 10x

X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
  split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = True, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
init = GlorotUniform(seed=42)
# preprocess data
X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
reg_name = 'l2'
l1_l = 0
l2_l = 0.01
opt = 'Nesterov'
step = 0.001
decay = 0.001
batch = 48
epoch = 40

CRNNopt_1 = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
with tf.device('/device:GPU:0'):
  # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
  history = CRNNopt_1.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=1, validation_data=(X_valid_cur, y_valid_norm_pcs))
# train_score = CRNNopt_1.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
val_acc = history.history['val_categorical_accuracy'][-1]
train_acc = history.history['categorical_accuracy'][-1]
print('reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data


(4430, 22, 900)
(16920, 22, 900)
(4230, 22, 900)
4430
16920
4230
Train on 16920 samples, validate on 4230 samples
Epoch 1/40
16920/16920 [==============================] - 17s 981us/sample - loss: 2.8610 - categorical_accuracy: 0.2819 - val_loss: 2.7611 - val_categorical_accuracy: 0.2931
Epoch 2/40
16920/16920 [==============================] - 12s 702us/sample - loss: 2.6251 - categorical_accuracy: 0.3459 - val_loss: 2.5288 - val_categorical_accuracy: 0.3546
Epoch 3/40
16920/16920 [==============================] - 12s 705us/sample - loss: 2.3958 - categorical_accuracy: 0.4150 - val_loss: 2.2764 - val_categorical_accuracy: 0.4430
Epoch 4/40
16920/16920 [==============================] - 12s 706us/sample - loss: 2.1394 - categorical_accuracy: 0.5058 - val_loss: 2.0810 - val_categorical_accuracy: 0.4787
Epoch 5/40
16920/16920 [==============================] - 12s 707us/sample - loss: 1.8685 - categorical_accuracy: 0.6014 - val_loss: 1.9761 - val_categorical_accuracy: 0.5059
Epoch 6/40


In [0]:
test_acc = CRNNopt_1.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
# print('reg name',reg_name,'reg value',reg_value,'opt name',opt_name,'batch size',batch_size,'epochs',epochs)
print('test_acc',test_acc)

4430/4430 - 2s - loss: 1.6867 - categorical_accuracy: 0.6562
test_acc [1.6867130012598317, 0.6562077]


In [0]:
predictions_2 = CRNNopt_1.predict(X_test_cur)

Ensemble from 2 Models

Final Accuracy

In [0]:
pred_final = (predictions_1 + predictions_2)/2
pred_label = np.array( np.argmax(pred_final, axis=1) )
true_label = np.array( np.argmax(y_test_norm_pcs, axis=1) )
acc_final = np.sum(pred_label == true_label) / y_test_norm_pcs.shape[0]
print('ensemble pred acc',acc_final)

ensemble pred acc 0.6593679458239278


# Ensemble with More Models 2x3
Model with the first set of parameters (3x copy)

In [0]:
# ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]
# ['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

pred_list_0 = []
# ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

size = 900
step = 10
end = int(size + step*10)    # gives 10x

X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
  split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = False, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
init = GlorotUniform(seed=42)
# preprocess data
X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
reg_name = 'l1'
l1_l = 0.001
l2_l = 0
opt = 'Nesterov'
step = 0.001
decay = 0.001
batch = 48
epoch = 40
num_copy = 3
for i in range(3):
  CRNNopt = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
  with tf.device('/device:GPU:0'):
    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
    history = CRNNopt.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_freq=[epoch], validation_data=(X_valid_cur, y_valid_norm_pcs))
  # train_score = CRNNopt.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
  val_acc = history.history['val_categorical_accuracy'][-1]
  train_acc = history.history['categorical_accuracy'][-1]
  print('reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data

  test_acc = CRNNopt.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
  print('test_acc',test_acc)
  predictions_i = CRNNopt.predict(X_test_cur)
  pred_list_0.append(predictions_i)

(4430, 22, 900)
(16920, 22, 900)
(4230, 22, 900)
4430
16920
4230
reg name l1 reg value 0.001 0 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.96501184 val_acc 0.6995272
4430/4430 - 12s - loss: 2.0256 - categorical_accuracy: 0.6474
test_acc [2.025575923058572, 0.6474041]
reg name l1 reg value 0.001 0 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.96217495 val_acc 0.68439716
4430/4430 - 2s - loss: 2.1920 - categorical_accuracy: 0.6357
test_acc [2.1919780456723803, 0.6356659]
reg name l1 reg value 0.001 0 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.9614657 val_acc 0.68723404
4430/4430 - 1s - loss: 2.0675 - categorical_accuracy: 0.6442
test_acc [2.067509276926114, 0.6442438]


In [0]:
# ensemble with only 3 of the same setting
pred_final = np.mean(np.array(pred_list_0), axis=0)
pred_label = np.array( np.argmax(pred_final, axis=1) )
true_label = np.array( np.argmax(y_test_norm_pcs, axis=1) )
acc_final = np.sum(pred_label == true_label) / y_test_norm_pcs.shape[0]
print('ensemble pred acc',acc_final)

ensemble pred acc 0.6776523702031603


Model with the second set of parameters (3x copy)

In [0]:
# ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]
# ['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

pred_list_1 = []
# ['size', 900, 'reg name', 'l2', 'reg value', 0, 0.01, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

size = 900
step = 10
end = int(size + step*10)    # gives 10x

X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
  split_data(X_train_valid, X_test, y_train_valid, y_test, test_split = False, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = True)
init = GlorotUniform(seed=42)
# preprocess data
X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)
reg_name = 'l2'
l1_l = 0
l2_l = 0.01
opt = 'Nesterov'
step = 0.001
decay = 0.001
batch = 48
epoch = 40
num_copy = 3
for i in range(3):
  CRNNopt = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
  with tf.device('/device:GPU:0'):
    # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
    history = CRNNopt.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_freq=[epoch], validation_data=(X_valid_cur, y_valid_norm_pcs))
  # train_score = CRNNopt.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
  val_acc = history.history['val_categorical_accuracy'][-1]
  train_acc = history.history['categorical_accuracy'][-1]
  print('reg name',reg_name,'reg value',l1_l,l2_l,'opt name',opt,'step',step,'decay',decay,'batch size',batch,'epochs',epoch)
  print('time window:',size,'train_acc',train_acc,'val_acc',val_acc)  # test on validation data

  test_acc = CRNNopt.evaluate(X_test_cur,  y_test_norm_pcs, verbose=2)
  print('test_acc',test_acc)
  predictions_i = CRNNopt.predict(X_test_cur)
  pred_list_1.append(predictions_i)

(4430, 22, 900)
(16920, 22, 900)
(4230, 22, 900)
4430
16920
4230
reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.97239953 val_acc 0.6877068
4430/4430 - 1s - loss: 1.6128 - categorical_accuracy: 0.6350
test_acc [1.6127599599937283, 0.6349887]
reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.97825056 val_acc 0.6900709
4430/4430 - 1s - loss: 1.6995 - categorical_accuracy: 0.6271
test_acc [1.699480424347096, 0.627088]
reg name l2 reg value 0 0.01 opt name Nesterov step 0.001 decay 0.001 batch size 48 epochs 40
time window: 900 train_acc 0.97730494 val_acc 0.68865246
4430/4430 - 1s - loss: 1.6263 - categorical_accuracy: 0.6535
test_acc [1.6263365437699224, 0.6534989]


In [0]:
# ensemble with only 3 of the same setting
pred_final = np.mean(np.array(pred_list_1), axis=0)
pred_label = np.array( np.argmax(pred_final, axis=1) )
true_label = np.array( np.argmax(y_test_norm_pcs, axis=1) )
acc_final = np.sum(pred_label == true_label) / y_test_norm_pcs.shape[0]
print('ensemble pred acc',acc_final)

ensemble pred acc 0.682392776523702


### Ensemble the 3+3 models to get the final testing accuracy

In [0]:
pred_list = pred_list_0 + pred_list_1
pred_final = np.mean(np.array(pred_list), axis=0)
pred_label = np.array( np.argmax(pred_final, axis=1) )
true_label = np.array( np.argmax(y_test_norm_pcs, axis=1) )
acc_final = np.sum(pred_label == true_label) / y_test_norm_pcs.shape[0]
print('ensemble pred acc',acc_final)

ensemble pred acc 0.7081264108352144


Accuracy after Ensemble for Each Subject

In [0]:
# Accuracy for each subject
for i in range(9): 
  print('Subject',i)
  index_test = list(filter(lambda x: person_test[x] == i, range(len(person_test))))
  pred_i = pred_final[index_test]
  true_i = y_test_norm_pcs[index_test]
  pred_label = np.array( np.argmax(pred_i, axis=1) )
  true_label = np.array( np.argmax(true_i, axis=1) )
  acc_final = np.sum(pred_label == true_label) / true_i.shape[0]
  print('Ensemble pred acc',acc_final)

Subject 0
Ensemble pred acc 0.64
Subject 1
Ensemble pred acc 0.56
Subject 2
Ensemble pred acc 0.72
Subject 3
Ensemble pred acc 0.8
Subject 4
Ensemble pred acc 0.5957446808510638
Subject 5
Ensemble pred acc 0.673469387755102
Subject 6
Ensemble pred acc 0.72
Subject 7
Ensemble pred acc 0.72
Subject 8
Ensemble pred acc 0.723404255319149


In [0]:
from google.colab import files
np.savetxt("bestCRNN_predict.csv", pred_final, delimiter=",")
files.download('bestCRNN_predict.csv')

# Train on Different Subjects

In [0]:
def pickUpOne(X_train_valid, X_test, y_train_valid, y_test, person_train_valid, person_test, index=0):
  index_train_valid = list(filter(lambda x: person_train_valid[x] == index, range(len(person_train_valid)))) 
  index_test = list(filter(lambda x: person_test[x] == index, range(len(person_test)))) 
  X_train_valid_first = X_train_valid[index_train_valid]
  y_train_valid_first = y_train_valid[index_train_valid]
  X_test_first = X_test[index_test]
  y_test_first = y_test[index_test]
  return X_train_valid_first, X_test_first, y_train_valid_first, y_test_first

In [0]:
Record_sub = []

for index in range(9):
  print('Working on subject',index)
  X_train_valid_sub, X_test_sub, y_train_valid_sub, y_test_sub = pickUpOne(X_train_valid, X_test, y_train_valid, y_test, person_train_valid, person_test, index)

  pred_list_all = []
  pred_list_all_sub = []
  # ['size', 900, 'reg name', 'l1', 'reg value', 0.001, 0, 'opt name', 'Nesterov', 'step', 0.001, 'decay', 0.001, 'batch size', 48, 'epochs', 40]

  size = 900
  step = 10
  end = int(size + step*10)    # gives 10x

  X_train_pcs_rsp, X_valid_pcs_rsp, X_test_pcs_rsp, y_train_norm_pcs, y_valid_norm_pcs, y_test_norm_pcs = \
    split_data(X_train_valid_sub, X_test_sub, y_train_valid_sub, y_test_sub, test_split = False, use_cv = True, cv = 5, size = size, step = step, end = end, printSize = False)
  init = GlorotUniform(seed=42)
  # preprocess data
  X_train_cur = np.expand_dims(X_train_pcs_rsp, axis=3)
  X_valid_cur = np.expand_dims(X_valid_pcs_rsp, axis=3)
  X_test_cur = np.expand_dims(X_test_pcs_rsp, axis=3)

  reg_name_list = ['l1','l2']
  l1_l_list = [0.001, 0]
  l2_l_list = [0, 0.01]
  opt = 'Nesterov'
  step = 0.001
  decay = 0.001
  batch = 48
  epoch = 40
  num_setting = 2
  num_copy = 3
  for k in range(num_setting):
    reg_name = reg_name_list[k]
    l1_l = l1_l_list[k]
    l2_l = l2_l_list[k]
    for i in range(3):
      print('Iteration of', k*num_copy+i+1, 'of',num_setting*num_copy)
      CRNNopt = crnn(init, time_period = size, l1_l = l1_l ,l2_l = l2_l, step = step, drop_ratio = 0.4, decay_r = decay, optim = opt)
      with tf.device('/device:GPU:0'):
        # validation_split=0.2 or validation_data=(X_valid_cur, y_valid_norm_pcs)
        history = CRNNopt.fit(X_train_cur, y_train_norm_pcs, epochs=epoch, batch_size=batch, verbose=0, validation_freq=[epoch], validation_data=(X_valid_cur, y_valid_norm_pcs))
      # train_score = CRNNopt.evaluate(X_train_valid_cur, y_train_valid_cur, verbose=0)
      val_acc = history.history['val_categorical_accuracy'][-1]
      train_acc = history.history['categorical_accuracy'][-1]
      test_acc = CRNNopt.evaluate(X_test_cur,  y_test_norm_pcs, verbose=0)

      print('train_acc',train_acc,'val_acc',val_acc,'test_acc',test_acc[-1])  # test on validation data

      predictions_i = CRNNopt.predict(X_test_cur)
      pred_list_all.append(predictions_i)

  # ensemble with 6 of the same setting
  pred_final = np.mean(np.array(pred_list_all), axis=0)
  pred_label = np.array( np.argmax(pred_final, axis=1) )
  true_label = np.array( np.argmax(y_test_norm_pcs, axis=1) )
  acc_final = np.sum(pred_label == true_label) / y_test_norm_pcs.shape[0]

  print('Ensemble Testing Acc within the Subject',acc_final)
  print('-'*20)
  Record_sub.append(pred_final)

Working on subject 0
Iteration of 1 of 6
train_acc 1.0 val_acc 0.70212764 test_acc 0.7
Iteration of 2 of 6
train_acc 0.9873684 val_acc 0.7170213 test_acc 0.76
Iteration of 3 of 6
train_acc 0.9994737 val_acc 0.71489364 test_acc 0.7
Iteration of 4 of 6
train_acc 0.9989474 val_acc 0.71914893 test_acc 0.68
Iteration of 5 of 6
train_acc 0.9831579 val_acc 0.72553194 test_acc 0.68
Iteration of 6 of 6
train_acc 0.9968421 val_acc 0.69361705 test_acc 0.66
Ensemble Testing Acc within the Subject 0.78
--------------------
Working on subject 1
Iteration of 1 of 6
train_acc 0.9968254 val_acc 0.4106383 test_acc 0.52
Iteration of 2 of 6
train_acc 0.994709 val_acc 0.4531915 test_acc 0.46
Iteration of 3 of 6
train_acc 0.9957672 val_acc 0.5042553 test_acc 0.36
Iteration of 4 of 6
train_acc 0.9851852 val_acc 0.48297873 test_acc 0.3
Iteration of 5 of 6
train_acc 0.9915344 val_acc 0.34893617 test_acc 0.4
Iteration of 6 of 6
train_acc 0.994709 val_acc 0.4574468 test_acc 0.48
Ensemble Testing Acc within the S

In [0]:
from google.colab import files
for i in range(9):
  np.savetxt("subBestCRNN_predict_"+str(i)+'.csv', Record_sub[i], delimiter=",")
  files.download("subBestCRNN_predict_"+str(i)+'.csv')